# LLMeter with Amazon SageMaker JumpStart

This notebook demonstrates how you can use LLMeter to measure latency and performance-under-load for LLMs hosted on [Amazon SageMaker JumpStart](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-jumpstart.html).

We start with the basics of invoking LLMs through LLMeter and running request batches - then discuss the higher-level *experiments* offered by LLMeter including latency heatmapping and ramp-up load testing.

## Setting Up

First, ensure your environment is set up with the required libraries to run the example. In addition to `llmeter` itself, we'll need:

- `sagemaker` - for looking up SageMaker Jumpstart configurations and working with model deployments
- `transformers` - for measuring and slicing text by token counts

In [ ]:
%pip install "llmeter[plotting]<1" "sagemaker>=2.224,<3" "transformers>=4.40,<5"

In [ ]:
# Python Built-Ins:
import json

# External Dependencies:
from llmeter.endpoints import SageMakerEndpoint, SageMakerStreamEndpoint
from llmeter.runner import Runner
from sagemaker.jumpstart.model import JumpStartModel
from transformers import AutoTokenizer
from upath import UPath  # Combined APIs for accessing cloud or local storage

Next, you'll need a deployed SageMaker endpoint to test against.

If you have on deployed already, fill in the `endpoint_name` below to use it.

Alternatively, this notebook can create one for you: Fill in a valid LLM `model_id` from the SageMaker JumpStart model hub or [this reference table](https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html). Examples include:

- `meta-textgeneration-llama-3-1-8b-instruct` (Instruction-tuned Llama 3.1 8B)
- `huggingface-llm-mistral-7b-instruct-v3` (Instruction-tuned Mistral 7B)
- ...and many more!

> ⚠️ **WARNING:** SageMaker Endpoints are billable resources charged by deployed instance-hour until the endpoint is deleted, and Large Language Models typically depend on high-performance accelerated compute.
>
> - Check out the [SageMaker pricing page](https://aws.amazon.com/sagemaker/pricing/) for details on how you'll be charged for deployed endpoints on different instance types
> - The automatic deployment code below will use the default instance type for your chosen model. For more control, consider manually creating your endpoint(s) through the [JumpStart model hub UI](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-jumpstart.html) or refer to the detailed docs on [deploying JumpStart models from Python](https://sagemaker.readthedocs.io/en/stable/overview.html#deploy-a-pre-trained-model-directly-to-a-sagemaker-endpoint).
> - Remember to **delete your endpoint(s)** when you're done experimenting - either from the [SageMaker Console](https://console.aws.amazon.com/sagemaker/home?#/endpoints) or referring to the *Clean up* section of this notebook.

In [ ]:
endpoint_name = None  # <-- Replace with an endpoint name or leave as `None` to create a new one
model_id = None  # <-- If creating a new endpoint, replace with a model ID from SageMaker JumpStart
model_version = "*" # <-- Replace with a specific version or leave `*` to use the latest one
accept_eula = True  # <-- some Jumpstart models requires explicitly accepting a EULA

In [ ]:
if endpoint_name is None:
    if model_id is None:
        raise ValueError("Either model_id or endpoint_name must be provided.")

    # Look up the provided model ID in the JumpStart hub:
    model = JumpStartModel(model_id=model_id, model_version=model_version)
    # Deploy with default settings (i.e. whatever this model's default instance type is):
    predictor = model.deploy(accept_eula=accept_eula, wait=True)
    endpoint_name = predictor.endpoint_name  # type: ignore

else:
    from sagemaker.jumpstart.session_utils import get_model_info_from_endpoint
    from sagemaker.predictor import retrieve_default

    # Connect to the existing endpoint:
    model_id, model_version, _, _, _ = get_model_info_from_endpoint(
        endpoint_name=endpoint_name
    )
    model = JumpStartModel(model_id=model_id, model_version=model_version)
    predictor = retrieve_default(endpoint_name=endpoint_name)

# SageMaker JumpStart models offer example payload(s) to illustrate their request format:
sample_payloads = [k.body for k in (model.retrieve_all_examples() or []) if k.body]
print(f"Got {len(sample_payloads)} sample payloads")
print("Sample payload 1:\n")
print(json.dumps(sample_payloads[0], indent=4))

# Try running the payload through the model to check it's deployed and working:
model_output = predictor.predict(sample_payloads[0])
print("Model output:\n")
print(json.dumps(model_output, indent=4))

Under the hood, LLMs represent text (and other media) in terms of discrete ["tokens"](https://en.wikipedia.org/wiki/Large_language_model#Tokenization) and in general response speed is strongly dependent on how many tokens are ingested and generated.

For our experiments, we'll need a **tokenizer**:

1. To automatically prepare input text payloads of different (approximate) token-lengths to help build up a "latency heatmap"
2. *In some cases*, to estimate the number of input tokens ingested and output tokens generated for each LLM request, since *some* JumpStart models don't provide this information
    - ℹ️ **Note:** Some LLM services (like Amazon Bedrock, and JumpStart models based on the [LMI/DJL container](https://docs.djl.ai/master/docs/serving/serving/docs/lmi/user_guides/lmi_input_output_schema.html#details)) actually include the consumed input & output token counts in each API response, and LLMeter will use this data wherever possible. Otherwise, it'll estimate the counts using your provided tokenizer function.

LLMeter does provide a ["dummy" tokenizer](../llmeter/tokenizers.py) by default which will split text on white-space (i.e. into "words"), but this gives a very rough approximation even in languages like English - and isn't really usable at all in languages (like Chinese, Japanese, Korean, Thai, etc) that don't separate words in this way.

In this example we'll use an open access LLama-3 tokenizer from Huggingface hub, but you can also use others like tiktoken or custom objects - as long as they provide `encode()` and `decode()` methods:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("philschmid/meta-llama-3-tokenizer")

## Basic inference and test runs

To measure the latency and throughput of the endpoint (including the network latency from the client where LLMeter is running to the actual endpoint), we start by creating a [`llmeter.Endpoint`](../llmeter/endpoints/base.py) object specifically for JumpStart real-time deployments.

In [ ]:
# Create an LLMeter Endpoint object for your deployed SageMaker endpoint:
sagemaker_endpoint = SageMakerEndpoint(
    endpoint_name,
    model_id=model_id,
    # The output path can be inferred from the example model output generated above:
    # (It may vary between different JumpStart models)
    generated_text_jmespath="[0].generated_text",
)

We can now verify that LLMeter is correctly capturing the output of the endpoint

In [ ]:
response = sagemaker_endpoint.invoke(payload=sample_payloads[0])
print(response)

You should see that:
- `time_to_last_token` (the overall response time) has been captured by LLMeter
- `time_to_first_token` is missing, because we didn't enable response streaming (see next section)
- for *most* models `num_tokens_input` and `num_tokens_output` (and therefore `time_per_output_token` too) will also be null - because the SageMaker endpoint didn't explicitly return input or output token counts.

Of course a single data point give us much confidence about typical endpoint performance.

We can use the [`Runner`](../llmeter/runner.py) class to set up a basic test run and calculate statistics.
- `clients` configures the number of concurrent (thread-based) clients that will send requests to the endpoint.
- `n_requests` is the total number of consecutive requests **each client** should perform (so the endpoint will receive clients * n_requests requests in total).
- `payload`s are the sample request(s) that should be used for the test. If `n_requests=None`, each client will just iterate through this list. Otherwise, the list will be shuffled to generate the required number of requests.
- `tokenizer` can be provided as a fallback to estimate the number of input & output tokens for APIs (like many SageMaker models) that don't explicitly report these counts.
- `output_path` can be a local path or `s3://...` URI, where the test results should be saved.

The `run()` method is an async function, and to execute it in a Jupyter notebook it requires to use the `await` keyword!

In [ ]:
endpoint_test = Runner(
    sagemaker_endpoint,
    tokenizer=tokenizer,
    output_path=f"outputs/{sagemaker_endpoint.model_id}",
)
results = await endpoint_test.run(payload=sample_payloads, n_requests=3, clients=3)

Once the test is completed, we can check the overall statistics aggregated from all the clients and requests:

In [ ]:
print(results)

We can also drill down to the individual responses if needed, as shown below.

Note that (unlike the single-execution example earlier) the token counts have been filled in with estimates from the provided tokenizer.

In [ ]:
results.responses

If the `output_path` was set, we can also find this information saved to files in the provided location:

In [ ]:
print(f"Contents of {results.output_path}\n----")
for sub in UPath(results.output_path).iterdir():
    print(sub)

## Streaming responses and Time-to-First-Token

Many LLM providers support *streaming* APIs which reduce user-perceived latency by sending the response in chunks rather than waiting for the whole generation to finish. In these contexts, the `time_to_first_token` is also a useful metric to understand how long consumers will have to wait before they can start to process the response.

If the selected Jumpstart endpoint supports model streaming, we can instead create an LLMeter `SageMakerStreamEndpoint` to handle this:

In [ ]:
sagemaker_endpoint_stream = SageMakerStreamEndpoint(endpoint_name, model_id=model_id)

To get a streaming output, we'll typically need to modify the payloads to include `"stream": true` - although the exact syntax may vary between different SageMaker JumpStart models:

In [ ]:
payloads_streaming = [{**k, "stream": True} for k in sample_payloads]
payloads_streaming

As before, we can first test that the `Endpoint` is correctly configured.

In [ ]:
response = sagemaker_endpoint_stream.invoke(payload=payloads_streaming[0])
print(response)

Once we're satisfied that the `Endpoint` is working as expected and we're now able to capture the `time_to_first_token` metric on the response - we can set up out test runner and `run()` it, as we did for the non-streaming case above.

In [ ]:
endpoint_stream_test = Runner(
    sagemaker_endpoint_stream,
    tokenizer=tokenizer,
    output_path=f"outputs/{sagemaker_endpoint_stream.model_id}",
)
results_stream = await endpoint_stream_test.run(
    payloads_streaming, clients=20, n_requests=5
)

We can view the summary statistics and individual responses as before, but now have a additional statistics to work with:

> ℹ️ **Note:** The `time_to_last_token` may differ between streaming and non-streaming invocations, so it's important to test the invocation method you'll actually be using. Don't choose to test streaming only because it provides additional statistics!

In [ ]:
print(results_stream)

In [ ]:
results_stream.responses

Now you've seen the core mechanics of running tests with LLMeter, let's explore some of the higher-level *experiments* that use these low-level runners under the hood.

## Mapping latency by input & output token counts

For many LLMs, the time to process a request can significantly depend on the length (in number of tokens) of the input provided and the output generated.

We can produce a heatmap showing how latency varies by these factors, to give an idea of how optimizing your input length or generation lengths might affect the response times observed by users.

The `LatencyHeatmap` experiment automatically generates a set of request payloads with varying (approximate) input lengths and uses it to test the endpoint.

To construct the requests, we need a base text to use as a seed. The semantic aspects are not particularly important, so any sufficiently long text can serve the purpose - but remember that many LLMs have their own internal guardrails, so it's possible that the model might decline to respond in some cases.

We'd like the generated reply to be limited by the `max_tokens` parameter (so the heatmap can measure latency for various output lengths), so will engineer a prompt that encourages the model to generate as long a response as possible from the seed text:

In [ ]:
def prompt_fn(prompt, **kwargs):
    formatted_prompt = f"Create a story based on the following prompt: {prompt}"
    return SageMakerStreamEndpoint.create_payload(formatted_prompt, **kwargs)

With a seed text and prompt generation function, we're ready to set up our latency heatmapping experiment.

- The `source_file` and `create_payload_fn` will be used to generate requests with various input lengths.
- The set of `input_lengths` you'd like to test is approximate, since in many cases the locally-available tokenizer won't exactly match the one used by the model
- The set of `output_lengths` you'd like to test may not always be reached, if the model stops generating early for the given prompts.
- The `requests_per_combination` impacts both the time to run the test and the quality of your output statistics. Note for example that it doesn't make sense to consider p95 or p99 latency on a dataset with only 10 requests!
- A higher number of concurrent `clients` will speed up the overall test run, but could cause problems if you reach quota limits (on as-a-service models) or high request volumes that start to impact response latency (see the "Load testing" section below for more details!)

Similar to low-level test Runners, the `output_path` can be used to configure where the test result data should be saved (either locally or on the Cloud).

In [ ]:
from llmeter.experiments import LatencyHeatmap

latency_heatmap = LatencyHeatmap(
    endpoint=sagemaker_endpoint_stream,
    clients=20,
    requests_per_combination=50,
    output_path=f"outputs/{sagemaker_endpoint_stream.model_id}/heatmap",
    source_file="MaryShelleyFrankenstein.txt",
    input_lengths=[50, 500, 1000],
    output_lengths=[128, 512, 1024],
    create_payload_fn=prompt_fn,
    tokenizer=tokenizer,
)

heatmap_results = await latency_heatmap.run()

print(heatmap_results)

(Assuming you installed LLMeter with the `plotting` extras, as shown at the start of this notebook), you'll be able to plot the heatmap results visually to explore how the latency varies with input and output token count:

In [ ]:
latency_heatmap.plot_heatmap()

For many models the overall `time_to_last_token` depends more strongly on the number of tokens *generated* by the model (`num_tokens_output`), while the `time_to_first_token` depends more strongly on the *input* length (`num_tokens_input`) if any significant correlation is present.

## Load testing with concurrent request ramp-up

If you're using a large-scale, as-a-service LLM provider like [Amazon Bedrock](https://aws.amazon.com/bedrock/), it's likely that the request volume for your use-case is insignificant compared to what the service handles overall - in which case you should check your [API quotas](https://docs.aws.amazon.com/bedrock/latest/userguide/quotas.html) are sufficient for your workload, but there's probably **limited value** in running actual load/volume tests.

However, if you're deploying an LLM on a service like Amazon SageMaker where you **size the underlying infrastructure** (number of compute instances, GPUs, etc) - then of course it will be important to understand how the latency of your model varies depending on the number of concurrent users / requests.

LLMeter's `LoadTest` experiment makes it easy to measure how latency and throughput change as a function of the number of requests.

As shown below, you configure:
- `sequence_of_clients` - the different concurrent request counts you'd like to measure latency & throughput for
- `n_requests` - the number of requests *each client* should send *in each test run* (i.e. the total number of requests received by the endpoint will be this multiplied by the sum of all the sequence_of_clients counts)

> ℹ️ In this example we'll use the same set of sample payloads as the previous sections, but since processing time is request- and response-dependent, you should ideally use payloads that are representative of your actual workload when load testing.

In [ ]:
from llmeter.experiments import LoadTest

sweep_test = LoadTest(
    endpoint=sagemaker_endpoint_stream,
    payload=payloads_streaming,
    sequence_of_clients=[1, 5, 20, 50, 100, 500],
    min_requests_per_client=5,
    min_requests_per_run=10, 
    output_path=f"outputs/{sagemaker_endpoint_stream.model_id}/sweep",
)
sweep_results = await sweep_test.run()

Detailed results are available for each concurrency level run in the load test:

In [ ]:
print(sweep_results[0])

...And (again assuming you installed LLMeter with the `plotting` extras, as shown at the start of this notebook), you can plot the output for a visual understanding of how the endpoint responds to the increase in the number of concurrent requests:

In [ ]:
sweep_test.plot_sweep_results()

Note the results are graphed here by `requests_per_minute` (throughput), so you might see the lines *turn back on themselves* if you reached a concurrent request/client count at which your endpoint became overwhelmed and processes fewer requests overall.

## Clean up

Since SageMaker endpoints are [priced](https://aws.amazon.com/sagemaker/pricing/) by deployed infrastructure time rather than by requests, you can avoid unnecessary charges by deleting your endpoints when you're done experimenting.

You can un-comment and run the cell below to delete the `endpoint_name` used in this notebook, and also review any endpoints you might have running in the [Inference > Endpoints section of the SageMaker Console](https://console.aws.amazon.com/sagemaker/home?#/endpoints)

In [ ]:
# from sagemaker.predictor import Predictor
# Predictor(endpoint_name).delete_endpoint()